In [26]:
import pandas as pd
import urllib3 as urllib
import json
import IPython.display

In [27]:
http = urllib.PoolManager()

def getResponse(url):
    operUrl = http.request('GET', url)
    if(operUrl.status==200):
        data = operUrl.data
        jsonData = json.loads(data.decode('utf-8'))
    else:
        print("Error receiving data", operUrl.getcode())
    return jsonData

In [28]:
ltc_data = getResponse('http://www.dph.illinois.gov/sitefiles/COVIDLTC.json')
reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])

In [29]:
#Saving a copy of source data 
ltc_data_json = json.dumps(ltc_data)

f = open("Source_data/IL_" + reporting_date + "LTC_data_Source.json","w")
f.write(ltc_data_json)
f.close()

In [30]:
df = pd.DataFrame(ltc_data['FacilityValues'])
df['reporting_date'] = reporting_date
df['% deaths'] = df['deaths'] / df['confirmed_cases']
df['num of facilities'] = 1 

In [31]:
df.to_csv('Reporting_data/IL_' + reporting_date + '_LTC_data_.csv', index = False)

In [42]:
a = df.sum()
print ('Date: %s' % reporting_date)
print ('Cases: %d' % a.confirmed_cases)
print ('Deaths:  %d' % a.deaths)
print ('Facilities: %d' % df.reporting_date.value_counts()[0])

Date: 2020-06-19
Cases: 21310
Deaths:  3609
Facilities: 593


In [33]:
df_county = df.groupby(by=['County']).sum()
df_county['% deaths'] = (df_county['deaths'] / df_county['confirmed_cases'])
df_county.sort_values('deaths', ascending=False)

,confirmed_cases,deaths,% deaths,num of facilities
County,,,,
Cook,12282,2092,0.170331,277
DuPage,2034,354,0.174041,59
Lake,1637,268,0.163714,47
Will,767,151,0.196871,30
Kane,893,145,0.162374,33
St. Clair,485,87,0.179381,14
McHenry,408,73,0.178922,12
Madison,249,56,0.224900,8
Winnebago,315,52,0.165079,18


In [34]:
import altair as alt
from vega_datasets import data
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O', sort='-x'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2

ModuleNotFoundError: No module named 'altair'

In [35]:
import altair as alt
from vega_datasets import data
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2 

ModuleNotFoundError: No module named 'altair'

In [36]:
df.sort_values(by=['deaths'], ascending=False).to_csv('IL_LTC_Covid.csv')

In [40]:
df.reporting_date.value_counts()[0]

593

In [22]:
df.head()

,County,FacilityName,confirmed_cases,deaths,status,reporting_date,% deaths,num of facilities
0,Boone,Park Place of Belvidere,6,0,Open,2020-06-12,0.000000,1
1,Boone,Symphony Northwoods,87,13,Open,2020-06-12,0.149425,1
2,Champaign,Champaign-Urbana Nursing and Rehab,2,0,Open,2020-06-12,0.000000,1
3,Champaign,Reflections Memory Care,12,1,Open,2020-06-12,0.083333,1
4,Champaign,University Rehab Center,5,0,Open,2020-06-12,0.000000,1


In [23]:
df.sum()

County               BooneBooneChampaignChampaignChampaignChristian...
FacilityName         Park Place of BelvidereSymphony NorthwoodsCham...
confirmed_cases                                                  20356
deaths                                                            3385
status               OpenOpenOpenOpenOpenOpenOpenOpenOpenOpenOpenOp...
reporting_date       2020-06-122020-06-122020-06-122020-06-122020-0...
% deaths                                                       81.6023
num of facilities                                                  580
dtype: object

In [22]:
df2 = pd.read_json(r'../Covid-County-LTC-Analysis/Source_data/IL_2020-05-29LTC_data_Source.json'[FacilityValues])

NameError: name 'FacilityValues' is not defined